In [1]:
from gensim.summarization import summarize
from gensim.summarization import keywords
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import re
import rouge


[nltk_data] Downloading package punkt to /home/visnja/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from pymongo import MongoClient

connection=MongoClient("mongodb://localhost:27017/crawler.contents")

db=connection.get_database()
articles = pd.DataFrame(list(db.contents.find()))


articles = articles.drop(columns=['visited','created_at','contentType','date','icon'])

In [3]:
def textrank(corpus, ratio=0.2):    
    try:
        s = summarize(corpus,ratio=ratio)
        return s
    except:
        return corpus                    

In [4]:
def clean(txt):
    txt = re.sub(r'\d+ min read', '', str(txt), flags=re.IGNORECASE)
    txt = re.sub(r'by reuters staff', '', str(txt), flags=re.IGNORECASE) 
    return txt

In [5]:
def evaluate_summary(y_test, predicted):    
   rouge_score = rouge.Rouge()    
   scores = rouge_score.get_scores(y_test, predicted, avg=True)       
   score_1 = round(scores['rouge-1']['f'], 2)    
   score_2 = round(scores['rouge-2']['f'], 2)    
   score_L = round(scores['rouge-l']['f'], 2)    
   print("rouge1:", score_1, "| rouge2:", score_2, "| rougeL:",
         score_2, "--> avg rouge:", round(np.mean(
         [score_1,score_2,score_L]), 2))

In [6]:
articles['text'] = articles['body'].map(lambda x:  " ".join(x) if type(x) is list else "")
articles["predicted"] = articles['text'].map(lambda x: clean(x)).map(lambda x: textrank(x,ratio=0.2) )

In [7]:

# for i,x in articles.iterrows():
#     try:
#         evaluate_summary(x["summary"],x["predicted"])
#     except Exception as e :
#         print(e)

In [4]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import sent_tokenize, word_tokenize

In [19]:
for i,x in articles.iterrows():
    db.contents.update_one({'_id':x['_id']},{'$set':{'summary_textrank': x['predicted']}})

In [6]:
articles[articles.summary.notnull()].apply((lambda x: sentence_bleu([word_tokenize(x['summary'])],word_tokenize(x['summary_textrank'])) ),axis=1).mean()

0.11994844245208319